### Librerías utilizadas

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from collections import Counter
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split

from sklearn.model_selection import RandomizedSearchCV,GridSearchCV

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.metrics import accuracy_score,roc_auc_score, roc_curve

### Datos utilizados

Se importan los datos que resultaron del proceso de preparación de datos:

In [2]:
data = pd.read_csv('C:/Users/gusta/OneDrive/Documentos/Tesis/Datos/Datos_prep.csv') 

Se seleccionan las variables que serán utilizadas como variables dependientes, estas variables son las que cuentan con un buen IV para ser considerados en el modelo:

In [3]:
var_predictivas = ['CC','VFA','CC_GRAS','CC_MUSC','CC_HUES','CC_AGUA','PAS',
                  'PAM','PESO','PAD','TALLA','GLU','IMC']
X = data[var_predictivas] 
y = data['TARGET_TRI'] 

### Separación de datos

Para evaluar mejor el rendimiento del modelo, es una buena estrategia dividir el conjunto de datos dos: un conjunto de entrenamiento (70%) y uno de prueba (30%). Sin embargo, dado que la muestra de datos no esta balanceada (17.53% de Eventos vs 82.47% de No-Eventos) se hará primero un submuestreo (undersampling) el cual consiste en eliminar aleatoreamente algunos registros pertenecientes a la clase de No-Eventos (0).

In [4]:
print('Original dataset shape %s' % Counter(y))

Original dataset shape Counter({0: 1479, 1: 278})


In [5]:
rus = RandomUnderSampler(random_state=1,sampling_strategy=0.25)
X_resampled, y_resampled = rus.fit_resample(X, y)
print('Resampled dataset shape %s' % Counter(y_resampled))

Resampled dataset shape Counter({0: 1112, 1: 278})


In [6]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled,y_resampled, 
                                                    test_size=0.2, random_state=1)

## Modelos supervisados para Clasificación

En esta sección se llevará a cabo la construcción de los modelos supervisados de Machine Learning que calculen la probabilidad de que el paciente tenga su nivel de Triglicéridos por arriba de los 150 mg/dL para posteriormente hacer la clasificación en Evento y No-Evento.

### Árbol de decisión

Se contruye un modelo de Árbol de Decisión usando Scikit-learn.

In [7]:
# Parámetros para el modelo
param_dict_tr= dict(criterion=['gini','entropy'], splitter=['best','random'],
                 max_depth=range(2,30), max_features=range(2,5),
                 max_leaf_nodes=range(5,35), min_samples_leaf=[0.05])

# Crear el objeto árbol de decisión para clasificación
tr_clf = DecisionTreeClassifier()

#Autotuning de los modelos
grid_tr = GridSearchCV(cv=5, estimator=tr_clf, n_jobs=-1, 
                     param_grid=param_dict_tr ,scoring='roc_auc')
grid_tr.fit(X_train,y_train)
tr_clf = grid_tr.best_estimator_

# Entrenar el árbol de decisión
tr_clf = tr_clf.fit(X_train,y_train)

### Random Forest

Se contruye un modelo de Random Forest usando Scikit-learn.

In [ ]:
# Parámetros para el modelo
param_dict_rf= dict(n_estimators=range(50,150), criterion=['gini','entropy'],
                 max_depth=range(2,30), max_features=range(2,5),
                 max_leaf_nodes=range(5,35), min_samples_leaf=[0.05])

# Crear el objeto Random Forest para clasificación
rf_clf = RandomForestClassifier()

#Autotuning de los modelos
grid_rf = GridSearchCV(cv=5, estimator=rf_clf, n_jobs=-1, 
                     param_grid=param_dict_rf ,scoring='roc_auc')
grid_rf.fit(X_train,y_train)
rf_clf = grid_rf.best_estimator_

# Entrenar el Random Forest
rf_clf = rf_clf.fit(X_train,y_train)

### Gradient Boosting

Se contruye un modelo de Gradient Boosting usando Scikit-learn.

In [ ]:
# Parámetros para el modelo
param_dict_gb= dict(criterion=['friedman_mse', 'mse', 'mae'], 
                 loss=['deviance','exponential'], learning_rate=np.arange(0.01,0.1,0.01)
                 max_depth=range(2,10), max_features=range(2,5),
                 max_leaf_nodes=range(5,35), min_samples_leaf=[0.05])

# Crear el objeto Gradient Boosting para clasificación
gb_clf = GradientBoostingClassifier()

#Autotuning de los modelos
grid_gb = GridSearchCV(cv=5, estimator=gb_clf, n_jobs=-1, 
                     param_grid=param_dict_gb ,scoring='roc_auc')
grid_gb.fit(X_train,y_train)
gb_clf = grid_gb.best_estimator_

# Entrenar el Gradient Boosting
gb_clf = gb_clf.fit(X_train,y_train)

### Estadísticos de precisión

A continuación, se muestra la curva ROC de cada uno de los modelos junto con estadístico AUC que indica el área debajo de la curva ROC.

In [ ]:
modelos = [{'label':'Decision Tree','modelo':tr_clf,},
           {'label':'Random Forest','modelo':rf_clf,},
           {'label':'Gradient Boosting','modelo':gb_clf,}]

plt.figure(figsize=(6,5))
for m in modelos:
    modelo = m['modelo']
    fpr, tpr, thresholds = roc_curve(y_test, modelo.predict_proba(X_test)[:,1])
    auc = roc_auc_score(y_test, modelo.predict(X_test))
    plt.plot(fpr,tpr,label='%s ROC (auc= %0.2f)' % (m['label'], auc))
plt.plot([0,1],[0,1],'r--')
plt.xlim([0.0,1.0])
plt.ylim([0.0,1.05]) 
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve (using validation data)')
plt.legend(loc="lower right")
plt.show()

La precisión de un modelo se calcula comparando los valores reales del conjunto de prueba y los valores predichos por el modelo.

In [ ]:
for m in modelos:
    modelo = m['modelo']
    acc= accuracy_score(y_test,modelo.predict(X_test))
    print('%s Accuracy: %0.2f' % (m['label'], acc))